In [84]:
from pyspark.sql.session import SparkSession
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import concat,concat_ws
from pyspark.sql.functions import *
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
import pandas as pd
import sklearn as sk
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [90]:
base_path='D:/PROJECTS/FASHIONPASS/cosin_similarity_registory/'
cs_tag_path=base_path+'cosine_similarity_cloection_tag_1_9_2023.csv'
cs_tag_collection_path=base_path+'cosine_similarity_cloection_tag_1_10_2023.csv'
cs_tag_collection_path_v1=base_path+'cosine_similarity_cloection_tag_v1_1_10_2023.csv'
cs_product_color=base_path+'cosine_similarity_product_color_1_9_2023.csv'
cs_product_title=base_path+'cosine_similarity_product_title_1_9_2023.csv'
product_path='D:/PROJECTS/FASHIONPASS/Database/product.csv'
cs_tag__collection_path=base_path+'cosine_similarity_cloection_tag_1_10_2023.csv'

In [86]:
spark = SparkSession.builder \
    .appName("PySpark MySQL Example") \
    .config("spark.driver.extraClassPath", "C:/spark/spark-3.5.0-bin-hadoop3/jars/mysql-connector-java-5.1.46.jar") \
    .getOrCreate()



In [91]:
cs_tag_df=spark.read.format('csv').option('header',True) \
.load(cs_tag_path)
cs_tag_collection_df=spark.read.format('csv').option('header',True) \
.load(cs_tag_collection_path) 

cs_tag_collection_v1_df=spark.read.format('csv').option('header',True) \
.load(cs_tag_collection_path_v1) 
cs_product_color_df=spark.read.format('csv').option('header',True) \
.load(cs_product_color)


cs_prodcuct_title_df=spark.read.format('csv').option('header',True) \
.load(cs_product_title)

Product_df=spark.read.format('csv').option('header',True) \
.load(product_path)

In [93]:
cs_tag_df=cs_tag_df.select(cs_tag_df.Product_A,cs_tag_df.Product_B,cs_tag_df.cosin_score.cast('float'))
cs_tag_collection_df=cs_tag_collection_df.select(cs_tag_collection_df.Product_A,cs_tag_collection_df.Product_B,cs_tag_collection_df.cosin_score.cast('float'))
cs_tag_collection_v1_df=cs_tag_collection_v1_df.select(cs_tag_collection_v1_df.Product_A,cs_tag_collection_v1_df.Product_B,cs_tag_collection_v1_df.cosin_score.cast('float'))
cs_product_color_df=cs_product_color_df.select(cs_product_color_df.Product_A,cs_product_color_df.Product_B,cs_product_color_df.cosin_score_color.cast('float'))
cs_prodcuct_title_df=cs_prodcuct_title_df.select(cs_prodcuct_title_df.Product_A,cs_prodcuct_title_df.Product_B,cs_prodcuct_title_df.cosin_score_title.cast('float'))

In [94]:
cs_prodcuct_title_df.show()

+---------+---------+-----------------+
|Product_A|Product_B|cosin_score_title|
+---------+---------+-----------------+
|        1|        2|        0.3454029|
|        1|        3|        0.3454029|
|        1|        4|       0.33695558|
|        1|        5|       0.50686985|
|        1|        6|        0.5391118|
|        1|        7|        0.5391118|
|        1|        8|        0.5391118|
|        1|        9|       0.54921216|
|        1|       10|       0.45818448|
|        1|       11|       0.45818448|
|        1|       12|       0.47669756|
|        1|       13|       0.28714523|
|        1|       14|        0.5835984|
|        1|       15|        0.5232813|
|        1|       16|        0.4204615|
|        1|       17|        0.4204615|
|        1|       18|        0.4204615|
|        1|       19|        0.4204615|
|        1|       20|        0.4204615|
|        1|       21|        0.4204615|
+---------+---------+-----------------+
only showing top 20 rows



In [7]:
def duplicate_col(data):
  df_cols = data.columns
  duplicate_col_index = [idx for idx,
  val in enumerate(df_cols) if val in df_cols[:idx]]
 
# Create a new list by renaming duplicate 
# columns by adding prefix '_duplicate_'+index
  for i in duplicate_col_index:
    df_cols[i] = df_cols[i] + '_duplicate_'+ str(i)
 
# Rename the duplicate columns in data frame
  data = data.toDF(*df_cols)
 
# Display the data frame
  return data

In [ ]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [20]:
# cs_tag_title_df=cs_tag_df.join(cs_prodcuct_title_df,cs_tag_df.Product_A==cs_prodcuct_title_df.Product_A,'left')
# cs_tag_title_df=duplicate_col(cs_tag_title_df)
# cs_tag_title_color_df=cs_tag_title_df.join(cs_product_color_df,cs_tag_title_df.Product_A==cs_product_color_df.Product_A,'left')

In [120]:
product_title='DARYA OVERSIZED CREWNECK PULLOVER'
selected_product_id=Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]
cs_prodcuct_title_df_product=cs_prodcuct_title_df.filter(cs_prodcuct_title_df['Product_A']==selected_product_id)
# cs_product_color_df_product=cs_product_color_df.filter(cs_product_color_df['Product_A']==selected_product_id)
cs_tag_df_product=cs_tag_df.filter(cs_tag_df['Product_A']==selected_product_id)
cs_tag_collection_df_product=cs_tag_collection_df.filter(cs_tag_collection_df['Product_A']==selected_product_id)
cs_tag_collection_v1_df_product=cs_tag_collection_v1_df.filter(cs_tag_collection_v1_df['Product_A']==selected_product_id)
# product_title=[title[0] for title in cs_prodcuct_title_df_product.select('Product_B').collect() ]
# product_title_color= [  product_color[0] for product_color in cs_product_color_df_product.filter(cs_product_color_df_product['Product_B'].isin(product_title)).select('Product_B').collect()]
# product_title_color_tag=[product_tag[0] for product_tag in cs_tag_df_product.filter(cs_tag_df_product['Product_B'].isin(product_title)).select('Product_B').collect()]
recomendation_df_product=cs_tag_df_product#cs_tag_df_product[cs_tag_df_product['Product_B'].isin(product_title_color_tag)]
link=[]
for row in recomendation_df_product.collect():
     if float(row['cosin_score'])>=.80:
        id=row['Product_B']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
             link.append(id)
        except:
            print('valid string')    
print(','.join(link))
            
recomendation_df_product=cs_tag_collection_df_product#cs_tag_df_product[cs_tag_df_product['Product_B'].isin(product_title_color_tag)]
link1=[]
for row in recomendation_df_product.collect():
       if float(row['cosin_score'])>=.85:
        id=row['Product_B']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
             link1.append(id)
        except:
            print('valid string')  

recomendation_df_product=cs_tag_collection_v1_df_product#cs_prodcuct_title_df_product#cs_tag_df_product[cs_tag_df_product['Product_B'].isin(product_title_color_tag)]
link3=[]
for row in recomendation_df_product.collect():
       if float(row['cosin_score'])>=.90:
        id=row['Product_B']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
             link3.append(id)
        except:
            print('valid string')              
print(','.join(link3))

recm=intersection(link,link1)
recm=intersection(recm,link3)
print(','.join(recm))

8692,2178,8687,1087,5312,4656,1385,8599,2072,8058,41,6085,6054,7860,3047,530,345,6707,7865,1533,1149,3164,3655,3056,3361,941,1443,3456,3438,8451,5435,3144,297,8450,4100,8066,4920,176,3236,8535,1033,3398,3870,615,2073,5174,751,2007,5146,2050,865,33,6674,2736,277,6511,2023,6671,6377,1173,3862,4909,3897,4675,3834,3101,3234,3225,2544,2053,4965,8554,3041,2724,7127,2101,2829,1867,3275,803,1015,6499,358,8665,5031,4939,3439,6977,6569,8294,960,3645,2487,1862,463,6579,4890,3075,3990,603,2949,6862,2124,8612,6849,6424,3111,7840,1086,8441,7951,4578,4881,696,8522,4898,3893,333,7858,3292,602,6702,5108,8327,4918,8534,2848,3942,5047,2849,7330,4222,4057,196,1002,3966,4966,3215,5147,5176,4649,2838,4854,6706,522,4921,2078,3979,3120,5153,3093
8692,8687,1942,3910,1385,2072,8058,3951,8630,6054,3069,6465,1533,8174,3164,1149,941,3053,3438,8450,297,8066,4920,3236,2992,1033,3398,3870,7732,8525,615,8456,2073,2007,7859,865,3859,2986,33,6674,2736,277,2023,6671,1173,3897,4675,3101,3234,2053,4965,8554,2724,5409,8806,

In [150]:
# product_title='UNO BODYSUIT'
# selected_product_id=Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]
# cs_prodcuct_title_df_product=cs_prodcuct_title_df.filter(cs_prodcuct_title_df['Product_A']==selected_product_id)
# cs_product_color_df_product=cs_product_color_df.filter(cs_product_color_df['Product_A']==selected_product_id)
# cs_tag_df_product=cs_tag_df.filter(cs_tag_df['Product_A']==selected_product_id)
# product_title=[title[0] for title in cs_prodcuct_title_df_product.select('Product_B').collect() ]
# product_title_color= [  product_color[0] for product_color in cs_product_color_df_product.filter(cs_product_color_df_product['Product_B'].isin(product_title)).select('Product_B').collect()]
# product_title_tag=cs_prodcuct_title_df_product.filter(cs_prodcuct_title_df_product['Product_B'].isin(link))
